In [50]:
#Data handling
import pandas as pd
import numpy as np

#Data preprocessing
import os
import pickle
# from keras.models import load_model

#Data modeling
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_score, recall_score, f1_score, auc
from sklearn.pipeline import Pipeline
from keras.models import load_model
import warnings

from imblearn.over_sampling import RandomOverSampler, SMOTE

warnings.filterwarnings('ignore')


In [3]:
input_features =[
    'Date',
    'Location',
    'MinTemp',
    'MaxTemp',
    'Rainfall',
    'Evaporation',
    'Sunshine',
    'WindGustDir',
    'WindGustSpeed',
    'WindDir9am',
    'WindDir3pm',
    'WindSpeed9am',
    'WindSpeed3pm',
    'Humidity9am',
    'Humidity3pm',
    'Pressure9am',
    'Pressure3pm',
    'Cloud9am',
    'Cloud3pm',
    'Temp9am',
    'Temp3pm',
    'RainToday',
    'RainTomorrow',
    'RainfallTomorrow'
    ]

selected_features = [
                'MinTemp', 
                'MaxTemp', 
                'Evaporation',
                'Sunshine',
                'Wind9amu', 
                'Wind9amv', 
                'Wind3pmu', 
                'Wind3pmv', 
                'WindGustu',
                'WindGustv', 
                'Max_Pressure',
                'Min_Pressure',
                'Max_Humidity',
                'Min_Humidity',
                'Estado_Cielo',
                'Cluster',
                'Date'
            ]

In [4]:
def feat_eng(df):
    def cluster_cities(df):
        city_cluster = {
            'Adelaide': 10,
            'Albany': 1,
            'Albury': 4,
            'AliceSprings': 3,
            'BadgerysCreek': 2,
            'Ballarat': 8,
            'Bendigo': 8,
            'Brisbane': 9,
            'Cairns': 6,
            'Canberra': 4,
            'Cobar': 4,
            'CoffsHarbour': 9,
            'Dartmoor': 0,
            'Darwin': 7,
            'GoldCoast': 9,
            'Hobart': 12,
            'Katherine': 7,
            'Launceston': 12,
            'Melbourne': 8,
            'MelbourneAirport': 8,
            'Mildura': 0,
            'Moree': 9,
            'MountGambier': 0,
            'MountGinini': 4,
            'Newcastle': 2,
            'Nhil': 0,
            'NorahHead': 2,
            'NorfolkIsland': 5,
            'Nuriootpa': 10,
            'PearceRAAF': 1,
            'Penrith': 2,
            'Perth': 1,
            'PerthAirport': 1,
            'Portland': 0,
            'Richmond': 2,
            'Sale': 8,
            'Sydney': 2,
            'SydneyAirport': 2,
            'Townsville': 6,
            'Tuggeranong': 4,
            'Uluru': 3,
            'WaggaWagga': 4,
            'Walpole': 1,
            'Watsonia': 8,
            'Williamtown': 2,
            'Witchcliffe': 1,
            'Wollongong': 2,
            'Woomera': 10
            }
    
        cluster = df['Location'].map(city_cluster)
    
        return cluster
    

    def vector_coordinates(df, column, dir, speed, threshold=1e-6):
        direcciones = {
            'N': 0,
            'NNE': 22.5,
            'NE': 45,
            'ENE': 67.5,
            'E': 90,
            'ESE': 112.5,
            'SE': 135,
            'SSE': 157.5,
            'S': 180,
            'SSW': 202.5,
            'SW': 225,
            'WSW': 247.5,
            'W': 270,
            'WNW': 292.5,
            'NW': 315,
            'NNW': 337.5
        }
        velocidad = df[speed]
        df[column + 'u'] = -velocidad * np.sin(np.deg2rad(df[dir].map(direcciones)))
        df[column + 'v'] = -velocidad * np.cos(np.deg2rad(df[dir].map(direcciones)))
        
        # Comprobación condicional para valores cercanos a cero
        df[column + 'u'][np.abs(df[column + 'u']) < threshold] = 0
        df[column + 'v'][np.abs(df[column + 'v']) < threshold] = 0
        
        return df

    def definir_estado_del_cielo(row):
        if row['Cloud9am'] == 0 and row['Cloud3pm'] == 0:
            return 'Despejado'
        elif row['Cloud9am'] <= 4 and row['Cloud3pm'] <= 4:
            return 'Parcialmente Nublado'
        elif row['Cloud9am'] <= 7 and row['Cloud3pm'] <= 7:
            return 'Nublado'
        else:
            return 'Muy Nublado'
    
    df=vector_coordinates(df,'Wind9am','WindDir9am','WindSpeed9am')
    df=vector_coordinates(df,'Wind3pm','WindDir3pm','WindSpeed3pm')
    df=vector_coordinates(df,'WindGust','WindGustDir','WindGustSpeed')
    df['Max_Pressure'] = df[['Pressure9am', 'Pressure3pm']].max(axis=1)
    df['Min_Pressure'] = df[['Pressure9am', 'Pressure3pm']].min(axis=1)
    df['Max_Humidity'] = df[['Humidity9am', 'Humidity3pm']].max(axis=1)
    df['Min_Humidity'] = df[['Humidity9am', 'Humidity3pm']].min(axis=1)
    df['Estado_Cielo'] = df.apply(definir_estado_del_cielo, axis=1)
    df['Cluster'] = cluster_cities(df)

    df.drop(['Cloud3pm','Cloud9am','Humidity3pm','Humidity9am','WindSpeed9am','WindSpeed3pm','WindGustSpeed','WindDir9am','WindDir3pm','WindGustDir','Pressure3pm','Pressure9am','Temp3pm','Temp9am'], axis=1, inplace=True)
    df = df.replace([np.inf, -np.inf], 0)

    return df

In [53]:
def dataframe_normalized(df,features):
    df_normalized = df.copy()
    skystate = {
        'Despejado' :               0,
        'Parcialmente Nublado' :    1,
        'Nublado' :                 2,
        'Muy Nublado' :             3
    }
    df_normalized['RainToday'] = df_normalized['RainToday'].map({'No': 0, 'Yes': 1})
    df_normalized['RainTomorrow'] = df_normalized['RainTomorrow'].map({'No': 0, 'Yes': 1})
    df_normalized['Date'] = pd.to_datetime(df_normalized['Date'])
    df_normalized['Date']=df_normalized['Date'].dt.dayofyear
    df_normalized['Estado_Cielo'] = df_normalized['Estado_Cielo'].map(skystate)
    scaler = StandardScaler()
    df_normalized[features] = scaler.fit_transform(df_normalized[features])
    
    X = df_normalized[['Sunshine','Min_Humidity']]
    y = df_normalized['RainTomorrow']

    return X

In [9]:
clasificadorNN = load_model('ClasificadorNN.h5')

In [14]:
def clasificador(X_test_nnc):
    clasificadorNN = load_model('ClasificadorNN.h5')
    y_pred = clasificadorNN.predict(X_test_nnc)
    return y_pred

In [15]:
pipeline = Pipeline([
    ('Feature Engineering', FunctionTransformer(feat_eng)),
    ('df Normalized', FunctionTransformer(dataframe_normalized, kw_args={'features': selected_features})),
    ('Model', clasificadorNN)
])


In [40]:
df = pd.read_csv('./files/weatherAUS_sin_nan.csv', usecols=input_features)

In [41]:
df.columns.to_list()


['Date',
 'Location',
 'MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RainToday',
 'RainTomorrow',
 'RainfallTomorrow']

In [42]:
# df['Min_Humidity'] = df[['Humidity9am', 'Humidity3pm']].min(axis=1)
# #I want to divide in train test
# X = df[['Sunshine','Min_Humidity']]
# y = df['RainTomorrow']
# X = df.drop(['RainTomorrow','RainfallTomorrow'], axis=1)
# y = df['RainTomorrow']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [65]:
#I want to obtain only a sample of the data to use for the prediction and check wether the pipeline works
df_sample = df.sample(1, random_state=42)

In [66]:
df_sample['Date'] = pd.to_datetime(df_sample['Date'])
a=df_sample['Date'].dt.dayofyear

In [60]:
a

20356    213
Name: Date, dtype: int64

In [61]:
print('WindSpeed9am' in df_sample.columns)

True


In [62]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 20356 to 20356
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              1 non-null      datetime64[ns]
 1   Location          1 non-null      object        
 2   MinTemp           1 non-null      float64       
 3   MaxTemp           1 non-null      float64       
 4   Rainfall          1 non-null      float64       
 5   Evaporation       1 non-null      float64       
 6   Sunshine          1 non-null      float64       
 7   WindGustDir       1 non-null      object        
 8   WindGustSpeed     1 non-null      float64       
 9   WindDir9am        1 non-null      object        
 10  WindDir3pm        1 non-null      object        
 11  WindSpeed9am      1 non-null      float64       
 12  WindSpeed3pm      1 non-null      float64       
 13  Humidity9am       1 non-null      float64       
 14  Humidity3pm       1 no

In [44]:
X_train_sample

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
19459,2015-07-21,NorahHead,10.8,19.3,2.0,1.1,8.633333,N,52.0,NW,...,22.0,85.0,72.0,1032.9,1029.6,5.0,3.0,14.2,17.0,Yes


In [16]:
pipeline

Pipeline(steps=[('Feature Engineering',
                 FunctionTransformer(func=<function feat_eng at 0x000001F5A9196830>)),
                ('df Normalized',
                 FunctionTransformer(func=<function dataframe_normalized at 0x000001F5A9196C20>,
                                     kw_args={'features': ['MinTemp', 'MaxTemp',
                                                           'Evaporation',
                                                           'Sunshine',
                                                           'Wind9amu',
                                                           'Wind9amv',
                                                           'Wind3pmu',
                                                           'Wind3pmv',
                                                           'WindGustu',
                                                           'WindGustv',
                                                           'Max_Pressure',
                                                           'Min_Pressure',
                                                           'Max_Humidity',
                                                           'Min_Humidity',
                                                           'Estado_Cielo',
                                                           'Cluster',
                                                           'Date']})),
                ('Clasificador',
                 FunctionTransformer(func=<function clasificador at 0x000001F5A9196B90>))])

In [67]:
#I want to check if the pipeline works predicting the sample
pipeline.predict(df_sample)


# pipeline.fit(df_sample)

AttributeError: 'FunctionTransformer' object has no attribute 'predict'